In [4]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics import confusion_matrix
import json

from src.vae_models.vae_trainer import VAEFraudTrainer
from src.evaluation import FraudEvaluationMetrics

import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.8.0+cu128
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4070 SUPER


## 1. Configuration Setup

In [5]:
model_path = '../../models/vae_baseline.pth'

if os.path.exists(model_path):
    print("Loading pre-trained VAE baseline model...")
    
    # Load checkpoint (PyTorch 2.6+ requires weights_only=False for custom objects)
    checkpoint = torch.load(model_path, weights_only=False, map_location='cpu')
    
    # Extract results from checkpoint
    results = checkpoint['metrics']
    config = checkpoint['config']
    
    print("\n" + "="*60)
    print("PRE-TRAINED MODEL LOADED SUCCESSFULLY")
    print("="*60)
    print(f"\nModel Configuration:")
    print(f"  Input Dim: {config['input_dim']}")
    print(f"  Hidden Dim: {config['hidden_dim']}")
    print(f"  Latent Dim: {config['latent_dim']}")
    print(f"  Beta: {config['beta']}")
    
    print(f"\nTest Set Performance:")
    print(f"  Total Cost: ${int(results['test_cost']):,}")
    print(f"  Precision: {results['test_precision']:.3f}")
    print(f"  Recall: {results['test_recall']:.3f}")
    if 'test_pr_auc' in results:
        print(f"  PR-AUC: {results['test_pr_auc']:.4f}")
    
    print(f"\nOptimal Threshold: {checkpoint['optimal_threshold']:.6f}")
    print(f"Training Time: {checkpoint.get('timestamp', 'N/A')}")
    
    print("\n✓ You can now skip to Section 3 (Model Comparison)")
else:
    print(f"Model not found at {model_path}")
    print("Please run the training cell below to train a new model.")

Loading pre-trained VAE baseline model...

PRE-TRAINED MODEL LOADED SUCCESSFULLY

Model Configuration:
  Input Dim: 22
  Hidden Dim: 128
  Latent Dim: 16
  Beta: 1.0

Test Set Performance:
  Total Cost: $20,790
  Precision: 0.984
  Recall: 0.252

Optimal Threshold: 5.090549
Training Time: 2025-10-12 20:12:53

✓ You can now skip to Section 3 (Model Comparison)


### Option: Load Pre-trained Model (Skip Training)

If you want to skip training and load the pre-trained model, run this cell instead of the training cell below.

In [ ]:
# Training configuration
config = {
    # Data
    'data_path': '../../data/processed/creditcard_fe.csv',
    'drop_features': 'logreg_baseline',  # Use same features as logistic regression
    
    # Model architecture
    'hidden_dim': 128,
    'latent_dim': 16,
    'beta': 1.0,
    
    # Training
    'learning_rate': 1e-3,
    'epochs': 50,
    'batch_size': 256,
    
    # Business costs 
    'C_FP': 550,  # False positive cost (investigation)
    'C_FN': 110,  # False negative cost (missed fraud)
    
    # Hardware
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'random_seed': 42,
    
    # Save paths
    'model_save_path': '../../models/vae_baseline.pth',
    'results_save_path': '../../results/figures/vae_baseline_training.png'
}

print("Baseline VAE Configuration:")
print("=" * 50)
for key, value in config.items():
    print(f"{key:20s}: {value}")

Baseline VAE Configuration:
data_path           : ../../data/processed/creditcard_fe.csv
drop_features       : logreg_baseline
hidden_dim          : 128
latent_dim          : 16
beta                : 1.0
learning_rate       : 0.001
epochs              : 50
batch_size          : 256
C_FP                : 550
C_FN                : 110
device              : cuda
random_seed         : 42
model_save_path     : ../../models/vae_baseline.pth
results_save_path   : ../../results/figures/vae_baseline_training.png


## 2. Train VAE Model

In [7]:
# Load Logistic Regression baseline results for comparison
with open('../../results/baseline/logreg_results.json', 'r') as f:
    lr_results = json.load(f)

print("="*70)
print("LOGISTIC REGRESSION BASELINE LOADED")
print("="*70)
print(f"\nTest Set Performance:")
print(f"  Total Cost: ${lr_results['total_cost']:,}")
print(f"  FP Cost: ${lr_results['fp_cost_total']:,} ({lr_results['false_positives']} × $550)")
print(f"  FN Cost: ${lr_results['fn_cost_total']:,} ({lr_results['false_negatives']} × $110)")
print(f"\nConfusion Matrix:")
print(f"  TN: {lr_results['true_negatives']:,}")
print(f"  FP: {lr_results['false_positives']}")
print(f"  FN: {lr_results['false_negatives']}")
print(f"  TP: {lr_results['true_positives']}")

LOGISTIC REGRESSION BASELINE LOADED

Test Set Performance:
  Total Cost: $43,450
  FP Cost: $22,550 (41 × $550)
  FN Cost: $20,900 (190 × $110)

Confusion Matrix:
  TN: 284,274
  FP: 41
  FN: 190
  TP: 302
